In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np 
from vqls import *
from utils import *


In [2]:
func_out = {'sigmoid': sigmoid_t,'elu': elu_t, 'relu': relu_t, 'sin':sin_m}#'tanh': tanh_t,
func_list = list(func_out.keys())
func_dict = {'sigmoid': .0,'tanh': 1.0,'elu':.12, 'relu':.0, 'sin':2}

pnq = {'3':(3,8),
        '4':(4,16)}
columns=['function', 'nq', 'knot', 'training cost', 'time(s)', 'weights', 'RSS_h', 'RSS_q' ]
tot = {}
for flab, fun in func_out.items():
    for nq, knot in  pnq.values():
        file_path = f"results/results_gvqs_{flab}_{nq}.json"
        df = pd.read_json(file_path).drop(['Condition number', 'norm(yk)', 'training_cost', 'exe_time','RSS_h'], axis =1 )
        tmp = []
        for el in df["in_train_weights"]:
             tmp.append(el[-1])
        df["weights"] = tmp
        df = df.drop(["in_train_weights"], axis = 1)
        tot[f"{flab}{nq}"] = df 

In [3]:
rss=[]
for flab, fun in func_out.items():
    for nq in [3,4]:
        dl = tot[f"{flab}{nq}"]
        rmse_scal = []
        rmse_yc = []
        for i, el in dl.iterrows():
            
            lower = 0.
            upper = 1.

            f_i = func_dict[flab]
            scaled=False    
            label = flab
            n_step = pnq[f"{nq}"][1]
            x = np.arange(lower, upper + .03, (upper-lower)/n_step).tolist() 
            xx = np.linspace(lower, upper, n_step) ##inputs sampling in the interval 0,1
            y = [fun(value,f_i) for value in xx]

            #tck=splrep(x,y,k=1) #coeffs
            norm = np.linalg.norm(y)
            y = y / norm
            matrix,vector,v_norm = GeneralizedVQS_System(n_step,label,x,xx,scaled=scaled)

            vqls_circuit = VQLS(matrix,v_norm,nq,opt='COBYLA') 

            c = np.linalg.solve(matrix,vector)

            y_c=np.dot(matrix,c) #classic

            y_fq=[]
            q = vqls_circuit.solution(el['weights']).real
            y_q=np.dot(matrix,q) #hybrid
                            #Quantum

            for a in matrix:
                y_fq.append(vqls_circuit.direct_prod2(el['weights'],a))
            rmse_yc.append(math.sqrt(np.square(np.subtract(y_c,y_fq)).mean()))
            rmse_scal.append(math.sqrt(np.square(np.subtract(y,y_fq)).mean()))
        dl["rmse_scal"] = rmse_scal
        dl["rmse_yc"] = rmse_yc


In [4]:
tot.keys()

dict_keys(['sigmoid3', 'sigmoid4', 'elu3', 'elu4', 'relu3', 'relu4', 'sin3', 'sin4'])

In [5]:
dk= tot["sigmoid3"]
#dk['RSS_q'] = dk['RSS_q']/dk['knot']
print(dk.pivot( columns = [], values=['rmse_yc','RSS_q']).sort_values(["rmse_yc"]).to_markdown())

|       |   rmse_yc |    RSS_q |
|:------|----------:|---------:|
| (12,) |  0.183223 | 0.268566 |
| (22,) |  0.18553  | 0.275371 |
| (11,) |  0.185691 | 0.275849 |
| (3,)  |  0.185943 | 0.276599 |
| (23,) |  0.698045 | 3.89813  |
| (6,)  |  0.702102 | 3.94358  |
| (17,) |  0.702309 | 3.9459   |
| (13,) |  0.702592 | 3.94908  |
| (18,) |  0.703671 | 3.96122  |
| (0,)  |  0.703877 | 3.96354  |
| (8,)  |  0.704963 | 3.97578  |
| (10,) |  0.706013 | 3.98764  |
| (9,)  |  0.715782 | 4.09875  |
| (19,) |  0.719192 | 4.1379   |
| (4,)  |  0.719874 | 4.14575  |
| (20,) |  0.719874 | 4.14575  |
| (14,) |  0.721589 | 4.16553  |
| (15,) |  0.725029 | 4.20533  |
| (7,)  |  0.738071 | 4.35799  |
| (5,)  |  1.08453  | 9.40971  |
| (21,) |  1.08523  | 9.42185  |
| (2,)  |  1.08552  | 9.42683  |
| (1,)  |  1.08588  | 9.43312  |
| (16,) |  1.08667  | 9.44684  |
| (24,) |  1.08667  | 9.44684  |


In [6]:
flab = "sigmoid"
nq = 3
dl = tot[f"{flab}{nq}"]
dl["RSS_q"][3]


0.2765988787

In [7]:
rss=[]
res = {}

for flab, fun in func_out.items():
    funres = {}
    # vqs results
    for stp in [20]:
        nq = 1
        file_path = f"results/results_vqs_{flab}_{nq}_{stp}.json"
        dl = pd.read_json(file_path).drop(["exe_time","weights", "training_cost", "seed"], axis = 1)

        funres[f"vqs_{stp}"]= {"rmse":np.min(dl["rmse"]), "RSS": np.sqrt(dl["RSS_q"][np.argmin(dl["rmse"])]/40) }


    # gvqs results
    for nq in [3,4]:
        dl = tot[f"{flab}{nq}"]
        rmse_scal = []
        rmse_yc = []
        lower = 0.
        upper = 1.
        for i, el in dl.iterrows():
            

            f_i = func_dict[flab]
            scaled=False    
            label = flab
            n_step = pnq[f"{nq}"][1]
            x = np.arange(lower, upper + .03, (upper-lower)/n_step).tolist() 
            xx = np.linspace(lower, upper, n_step) ##inputs sampling in the interval 0,1

            #tck=splrep(x,y,k=1) #coeffs
            matrix,vector,v_norm = GeneralizedVQS_System(n_step,label,x,xx,scaled=scaled)

            vqls_circuit = VQLS(matrix,v_norm,nq,opt='COBYLA') 

            c = np.linalg.solve(matrix,vector)

            y_c=np.dot(matrix,c) /np.linalg.norm(vector)#classic
            
            y_fq=[]
            for a in matrix:
                y_fq.append(vqls_circuit.direct_prod2(el['weights'],a))
            rmse_yc.append(math.sqrt(np.square(np.subtract(v_norm,y_fq)).mean()))
        funres[f"gvqs_{nq}"]= {"rmse":np.min(rmse_yc),"nrsme": np.min(rmse_yc)/(np.max(y_c) -np.min(y_c))}
    res[flab] = funres


In [8]:
for key in res.keys():
    print('\n' + '='*15 + '  ' +key+ '  ' + '='*15)
    print(pd.DataFrame(res[key]).T)#.to_markdown(tablefmt= 'latex', floatfmt=".4f"))



===============  sigmoid  ===============
            rmse       RSS                nrsme
vqs_20  0.086756  0.086756                  NaN
gvqs_3  0.125112       NaN  0.23004417824751283
gvqs_4  0.077685       NaN  0.20026862677259832

===============  elu  ===============
            rmse       RSS                nrsme
vqs_20  0.154265  0.154265                  NaN
gvqs_3  0.140234       NaN   0.2159109992738178
gvqs_4  0.077421       NaN  0.16265677020301578

===============  relu  ===============
            rmse       RSS                nrsme
vqs_20  0.093531  0.093531                  NaN
gvqs_3  0.154328       NaN   0.2111459218949655
gvqs_4  0.078688       NaN  0.14444248732569634

===============  sin  ===============
            rmse       RSS                nrsme
vqs_20  0.333613  0.333613                  NaN
gvqs_3  0.109348       NaN    0.190176644899118
gvqs_4  0.063676       NaN  0.15519009998759212


In [38]:
func_out = {'sigmoid': sigmoid,'tanh': tanh,'elu': elu, 'relu': relu, 'sin':sin_m}
func_dict = {'sigmoid': .0,'tanh': 1.0,'elu':.12, 'relu':.0, 'sin':-1}
step = (upper-lower)/n_step
result = {}
n_step= 20
step = (1-(-1))/n_step
# function specific 
f_i = func_dict[label]
upper = 1 
lower = -1
#TODO THE FUCK IS FE
f_e = f_i

func = func_out[label]


In [39]:
#################################
x = np.arange(lower, upper + .03, step).tolist()
y = [func(value,f_i) for value in x]

print(x)
print(y)

[-1.0, -0.9, -0.8, -0.7000000000000001, -0.6000000000000001, -0.5000000000000001, -0.40000000000000013, -0.30000000000000016, -0.20000000000000018, -0.1000000000000002, -2.220446049250313e-16, 0.09999999999999964, 0.19999999999999973, 0.2999999999999998, 0.3999999999999997, 0.49999999999999956, 0.5999999999999996, 0.6999999999999997, 0.7999999999999996, 0.8999999999999995, 0.9999999999999996]
[0.5000000000000001, 0.6545084971874737, 0.7938926261462367, 0.9045084971874737, 0.9755282581475768, 1.0, 0.9755282581475768, 0.9045084971874738, 0.7938926261462368, 0.654508497187474, 0.5000000000000003, 0.34549150281252683, 0.20610737385376382, 0.09549150281252644, 0.024471741852423345, 0.0, 0.024471741852423012, 0.095491502812526, 0.20610737385376288, 0.3454915028125254, 0.4999999999999993]


In [45]:
X = []
for i in range(1, len(x)):
    X.append(np.arange(x[i - 1], x[i], step - 0.001).tolist())

print(X)

[[-1.0, -0.901], [-0.9, -0.801], [-0.8, -0.7010000000000001], [-0.7000000000000001, -0.6010000000000001], [-0.6000000000000001, -0.5010000000000001], [-0.5000000000000001, -0.40100000000000013], [-0.40000000000000013, -0.30100000000000016], [-0.30000000000000016, -0.20100000000000015], [-0.20000000000000018, -0.10100000000000017], [-0.1000000000000002, -0.0010000000000001952], [-2.220446049250313e-16, 0.09899999999999978], [0.09999999999999964, 0.19899999999999965], [0.19999999999999973, 0.2989999999999997], [0.2999999999999998, 0.3989999999999998], [0.3999999999999997, 0.49899999999999967], [0.49999999999999956, 0.5989999999999995], [0.5999999999999996, 0.6989999999999996], [0.6999999999999997, 0.7989999999999997], [0.7999999999999996, 0.8989999999999996], [0.8999999999999995, 0.9989999999999994]]


In [54]:
x1 = [item for sublist in X for item in sublist]
y1 = [func(value,f_e) for value in x]

print(len(X))
x2 = np.array([i for i in x1]).reshape([20,2])

print([(a[0]- b) for a, b in zip(x2,x)])

20
[tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True), tensor(0., requires_grad=True)]
